In [1]:
import argparse
import sys
import os
import time
import datetime
import ast
from collections import Counter
from pathlib import Path
import hydra
import copy

import numpy as np
import torch

# Ensure local src/ is importable
sys.path.insert(0, str(Path(os.getcwd()).parent / "src"))

from my_genQC.inference.eval_metrics import UnitaryFrobeniusNorm, UnitaryInfidelityNorm
from my_genQC.inference.evaluation_helper import get_unitaries, get_srvs
from my_genQC.inference.sampling import generate_compilation_tensors, generate_tensors, decode_tensors_to_backend
from my_genQC.pipeline.diffusion_pipeline import DiffusionPipeline
from my_genQC.platform.simulation import Simulator, CircuitBackendType
from my_genQC.platform.tokenizer.circuits_tokenizer import CircuitTokenizer
from my_genQC.utils.misc_utils import infer_torch_device, get_entanglement_bins
from my_genQC.dataset import circuits_dataset
from my_genQC.models.config_model import ConfigModel
from my_genQC.utils.config_loader import load_config, store_tensor, load_tensor

from quantum_diffusion.data.dataset import DatasetLoader
from quantum_diffusion.training.training import DiffusionTrainer

In [2]:
# Config loading
with hydra.initialize(version_base=None, config_path="../conf"):
    cfg = hydra.compose(config_name="config.yaml", overrides=["training=cloob_srv"])
    cfg = cfg["training"]

In [3]:
trainer = DiffusionTrainer(config=cfg, device="cuda")
dataset_loader = DatasetLoader(config=cfg, device="cuda")

dataset = dataset_loader.load_dataset(cfg.general.dataset, load_embedder=True)

2026-01-03 15:43:44 - quantum_diffusion.data.dataset - INFO - Detected multiple datasets in dataset_path. Loading all and combining them...
[INFO]: Loading tensor from `../datasets/paper_quditkit/srv_8q_dataset/dataset/ds_x.safetensors` onto device: cuda.
[INFO]: Loading tensor from `../datasets/paper_quditkit/srv_8q_dataset/dataset/ds_y.safetensors` onto device: cuda.
[INFO]: Instantiated config_dataset from given config on cuda.
[WARNING]: The value 0 is reserved for background tokens, i.e. qubit time position which are not effected by gates.
[WARNING]: Automatically incrementing all vocabulary values by one ...


/usr/local/lib/python3.11/dist-packages/open_clip/factory.py:450: UserWarning: QuickGELU mismatch between final model config (quick_gelu=False) and pretrained tag 'openai' (quick_gelu=True).
  warnings.warn(


[INFO]: `my_genQC.models.frozen_open_clip.CachedFrozenOpenCLIPEmbedder` instantiated from given `config` on cuda.
[INFO]: `my_genQC.models.frozen_open_clip.CachedFrozenOpenCLIPEmbedder`. Found no key `save_type` in `config`. No state dict loaded.
[INFO]: `my_genQC.models.frozen_open_clip.CachedFrozenOpenCLIPEmbedder`. Freeze model: True
2026-01-03 15:43:49 - quantum_diffusion.data.dataset - INFO - Loading local text embedder weights...
2026-01-03 15:43:49 - quantum_diffusion.data.dataset - WARNING - Only CLOOB weights are loadable!
2026-01-03 15:43:55 - quantum_diffusion.data.dataset - WARNING - 0 missing keys, first 10: []
2026-01-03 15:43:55 - quantum_diffusion.data.dataset - WARNING - 339 unexpected keys, first 10: ['visual.conv1.weight', 'visual.bn1.weight', 'visual.bn1.bias', 'visual.bn1.running_mean', 'visual.bn1.running_var', 'visual.bn1.num_batches_tracked', 'visual.conv2.weight', 'visual.bn2.weight', 'visual.bn2.bias', 'visual.bn2.running_mean']
2026-01-03 15:43:55 - quantum_d

  0%|          | 0/6 [00:00<?, ?it/s]

 - balance_tensor_dataset, njobs=1, number of samples=599936
 - uniquify_tensor_dataset, number of samples now 599936
 - balancing


...

 - dataset size after balancing 497496
 - balance_tensor_dataset, njobs=1, number of samples=281970
 - uniquify_tensor_dataset, number of samples now 281970
 - balancing


  0%|          | 0/12 [00:00<?, ?it/s]

 - dataset size after balancing 281970
 - balance_tensor_dataset, njobs=1, number of samples=163327
 - uniquify_tensor_dataset, number of samples now 163327
 - balancing


  0%|          | 0/5 [00:00<?, ?it/s]

 - dataset size after balancing 163327
Split: Train 2415492 - Test 127131 

2026-01-03 15:44:41 - quantum_diffusion.data.dataset - INFO - Datasets combined into a mixed dataset


In [4]:
trainer.setup_model(dataset=dataset, text_encoder=dataset_loader.text_encoder, tokenizer=dataset_loader.tokenizer)

2026-01-03 15:44:53 - quantum_diffusion.training.training - INFO - Setting up diffusion model...
[INFO]: `genQC.models.unet_qc.QC_Cond_UNet` instantiated from given `config` on cuda.
[INFO]: `genQC.models.unet_qc.QC_Cond_UNet`. No `save_path` provided. Found no key `save_path` in `config`. No state dict loaded.
[INFO]: `genQC.models.unet_qc.QC_Cond_UNet`. Freeze model: False
2026-01-03 15:44:53 - quantum_diffusion.training.training - INFO - Model setup completed successfully
